### Numpy code

In [2]:
import numpy as np

# Example: Large matrices (adjust size as needed)
n = 7000  # For very large matrices, ensure you have enough RAM
A = np.random.rand(n, n).astype(np.float32)
B = np.random.rand(n, n).astype(np.float32)

C = np.dot(A, B)  # warm-up and Matrix multiplication

%timeit -r 2 -o np.dot(A, B)

print(f"Result shape: {C.shape}")
print(f"Result type: {C.dtype}")


1.41 s ± 42.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result shape: (7000, 7000)
Result type: float32


In [3]:
import torch
import time

# 1. Comprobar si la GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# 2. Convertir arrays de NumPy a Tensores de PyTorch y moverlos a la GPU
# Es vital usar .to(device) para que el cálculo no se quede en la CPU
A_torch = torch.from_numpy(A).to(device)
B_torch = torch.from_numpy(B).to(device)

# 3. Función de multiplicación
def matmul_pytorch(a, b):
    return torch.matmul(a, b)

# 4. Medición de tiempo (Sincronizando la GPU)
# Al igual que con CuPy, debemos esperar a que la GPU termine para medir bien
torch.cuda.synchronize()
start = time.time()

C_torch = matmul_pytorch(A_torch, B_torch)

torch.cuda.synchronize()
end = time.time()

print(f"Resultado en PyTorch: {C_torch.shape}")
print(f"Tiempo en GPU con PyTorch: {(end - start):.4f} segundos")

# Opcional: Si necesitas el resultado de vuelta en NumPy/CPU
# C_result = C_torch.cpu().numpy()

Usando dispositivo: cuda


/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 1080 which is of cuda capability 6.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 1080 with CUDA capability sm_61 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 1080 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  wa

Resultado en PyTorch: torch.Size([7000, 7000])
Tiempo en GPU con PyTorch: 0.3725 segundos


Análisis de la actividad extra (PyTorch):

La multiplicación de matrices es una operación altamente paralelizable. Mientras que la CPU procesa las filas y columnas de forma secuencial o con hilos limitados, la GPU descompone la matriz 7000×7000 en miles de pequeñas operaciones que se ejecutan simultáneamente en los núcleos CUDA.

En mis pruebas, PyTorch en GPU ha demostrado ser significativamente más rápido que NumPy. Además, la integración de PyTorch con el ecosistema de Python es tan fluida que el cambio de código es mínimo, lo que explica su dominio en el campo del Deep Learning.